In [0]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sys


In [0]:
# 시드값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(3)

In [0]:
# 생성자 모델 생성
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2))) #128은 임의로 정한 노드의 수임, 7*7 이미지의 최초 크기, input_dim=100 100차원의 랜덤 벡터 준비
generator.add(BatchNormalization()) # 배치 정규화
generator.add((Reshape(7, 7, 128))) # 컨볼루션 레이어가 받아들일 수 있는 형태로 바꾸어주는 코드
generator.add(UpSampling2D()) # 이미지의 가로, 세로 크기를 2배씩 늘려줌 => 14*14 크기로 늘어남
generator.add(Conv2D(64, kernel_size=3, padding='same')) # padding='same' 모자라는 부분은 0으로 채워줌
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2))) # 불안정한 ReLu 말고 변형한 LeakyReLu, 0보다 작을 경우 0.2를 곱해라
# LeakyReLU()함수는 ReLU() 함수에서 x값이 음수이면 무조건 0이 되어 뉴런들이 일찍 소실되는 단점을 보완하기 위해 0이하에서도 작은 값을 갖게 만드는 활성화 함수임
generator.add(UpSampling2D()) # 28*28 크기로 늘어남 (작은 크기의 이미지를 점점 늘려가면서 컨볼루션 레이어를 지나치게 하는 것이 DCGAN의 특징)
generator.add(Conv2D(64, kernel_size=5, padding='same', activation='tanh')) # 한 번 더 컨볼루션 과정을 거친 후 판별자에게 값을 넘길 준비

# 판별자 부분은 컨볼루션 신경망 모델을 그대로 가져와서 사용하면 됨
# 판별자는 진짜인지 가짜인지 판별만 해줄 뿐, 자기 자신이 학습하는 데 쓰이는 게 아니라 생성자로 넘겨 주어 생성자가 업데이트된 이미지를 만들도록 해야 함
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape(28, 28, 1), padding='same')) # strides 마스크를 몇 칸씩 이동시킬지 옵션(default=1), 가로 세로 크기가 더 줄어 새로운 특징을 뽑아주는 효과 작용(새로운 필터를 적용한 효과)
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten()) # 가로 세로 2차원을 1차원으로 변환
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable=False # 가중치를 저장하는 학습 기능 끄기

In [0]:
# 생성자와 판별자 연결(출력 진위 여부 가리기)

g_input = Input(shape(100,)) # 랜덤 100개 벡터를 keras의 Input() 함수에 집어넣어 생성자에 입력할 g_input 생성
dis_output = discriminator(generator(g_input)) # 28*28 크기의 이미지가 그대로 판별자 모델 discriminator()의 입력값으로 들어감, 판별=0/1

# gan 모델 컴파일 (생성자와 판별자 연결)
gan = Model(g_input, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [0]:
# 조기 종료 설정
# from keras.callbacks import EarlyStopping
# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [0]:
# 실행 함수 선언
def gain_train(epoch, batch_size, saving_interval):

  # MNIST 데이터 불러오기
  # MNIST 데이터 다시 불러와서 이용(테스트 과정 필요 없음, 이미지만 사용 - x_train만 호출)
  (x_train, _), (_, _) = mnist.load_data()

  # 가로 28, 세로 28 픽셀, 흑백1, 실수 변환
  x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')

  # 0-255 사이의 픽셀 값에서 127.5를 뺀후 127.5로 나눠줌 (tanh() 함수 사용 전에 -1~1 사이의 값으로 만들어줌)
  x_train = (x_train - 127.5) / 127.5

  # batch_size만큼 mnist 손글씨 이미지 랜덤으로 불러와서 판별자에 집어넣음
  # 실제 이미지 (1 입력)
  true = np.ones((batch_size, 1))
  idx = np.random.randint(0, x_train.shape[0], batch_size)
  imgs = x_train[idx]
  d_loss_real = discriminator.train_on_batch(imgs, true)

In [0]:
# 학습 결과 출력
accuracy = model.predict(x_test, y_test)[1]
loss = model.predict(x_test, ytest)[0]

print('accuracy=', accuracy, ', loss=', loss)

In [0]:
# 모델 학습 결과 시각화
# import matplotlib.pyplot as plt

# 테스트셋 오차
y_vloss = history.history['val_loss']

# 학습셋 오차
y_loss = history.history['loss']

# 그래프 설정 - 1
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker'.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker'.', c='blue', lable='Trainset_loss')

# 그리드, 레이블 설정
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

# 테스트셋 정확도
y_vacc = history.history['val_acc']

# 학습셋 정확도
y_acc = history.history['acc']

# 그래프 설정 - 2
x_len = numpy.arange(len(y_acc))
plt.plot(x_len, y_vacc, marker'.', c='red', label='Testset_acc')
plt.plot(x_len, y_acc, marker'.', c='blue', lable='Trainset_acc')

# 그리드, 레이블 설정
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()